In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score
import os
import dill
from tensorflow.keras.callbacks import EarlyStopping
import scipy

dill.dump_session('notebook_env2.db')
dill.load_session('notebook_env.db')

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

Using TensorFlow backend.
C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\pc\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\pc\AppData\Roaming\Python\

In [2]:
batch_size=32
train_datagen = ImageDataGenerator() 
valid_datagen = ImageDataGenerator() 
test_datagen = ImageDataGenerator() 



test_generator = test_datagen.flow_from_directory(
    directory=r"D:/BACHDataset/SegTest",
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
    seed=42
)



Found 40 images belonging to 4 classes.


In [3]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import tensorflow as tf


In [4]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [5]:

model = []
model1 = keras.models.load_model('D:\\BACHWeights\\DenseNetAugSeg\\my_model',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
model2 = keras.models.load_model('D:\\BACHWeights\ResNet50AugSeg\\my_model',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})
model3 = keras.models.load_model('D:\\BACHWeights\ResNet101AugSeg\\my_model',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})

model.append(model1)
model.append(model2)
model.append(model3)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:


models = []
for i in range(len(model)):

    models.append(model[i])



# #Predictions
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()






from sklearn.metrics import classification_report, confusion_matrix
    
# Predict labels with models
labels = []
for m in models:
    pred=m.predict_generator(test_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
    predicts=np.argmax(pred,axis=-1)
    labels.append(predicts)
    #print(pred)

    #Classification Report

    print(predicts)
    print('Confusion Matrix')
    print(confusion_matrix(test_generator.classes,predicts))
    print('Classification Report')
    target_names = ['Bengin', 'InSitu', 'Invasive','Normal']
    print(classification_report(test_generator.classes, predicts, target_names=target_names))
  
    





    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=1)[0]
labels = np.squeeze(labels)


filenames=test_generator.filenames
pd.DataFrame({'ImageId' : filenames, 'Label' : labels }).to_csv("D:\\BACHWeights\\results.csv", index=False)




40/40 [==============================] - 16s 392ms/step
[0 0 0 1 0 0 3 2 0 0 1 1 0 1 1 1 1 1 1 1 3 2 2 2 2 2 0 2 2 2 3 3 1 3 3 3 3
 3 3 3]
Confusion Matrix
[[7 1 1 1]
 [1 9 0 0]
 [1 0 8 1]
 [0 1 0 9]]
Classification Report
              precision    recall  f1-score   support

      Bengin       0.78      0.70      0.74        10
      InSitu       0.82      0.90      0.86        10
    Invasive       0.89      0.80      0.84        10
      Normal       0.82      0.90      0.86        10

    accuracy                           0.82        40
   macro avg       0.83      0.83      0.82        40
weighted avg       0.83      0.82      0.82        40

40/40 [==============================] - 13s 337ms/step
[0 1 0 0 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 2 2 1 2 3 2 2 2 2 3 3 3 3 3 3 3
 3 3 3]
Confusion Matrix
[[ 8  2  0  0]
 [ 1  9  0  0]
 [ 1  1  7  1]
 [ 0  0  0 10]]
Classification Report
              precision    recall  f1-score   support

      Bengin       0.80      0.80      0.80     